In [2]:
from biscuit_functions import *
from ipywidgets import interactive, FloatSlider
from scipy.signal import find_peaks

In [3]:
dunky_df = pd.read_csv('datasets/dunking-data.csv')
dunky_df = washburn_eqn(dunky_df)

In [6]:
max_bin, min_bin, bw_steps = 1.0, 0.1, 0.01
min_tolerance, max_tolerance, tolerance_steps = 1e6, 4.5e6, 0.5e6

bw_values = np.arange(min_bin, max_bin + bw_steps, bw_steps)
tolerance_values = np.arange(min_tolerance, max_tolerance + tolerance_steps, tolerance_steps)

In [9]:
def get_all_peak_tol_evolutions(datapoints, bw_values):
    store_all_evolutions = {}

    for tol in tolerance_values:
        store_all_evolutions[tol] = follow_peak_evolution(datapoints, bw_values, tol)
        
    return store_all_evolutions

storage_oftols = {}
for biscuit in dunky_df['biscuit'].unique():
    datapoints = dunky_df[dunky_df['biscuit'] == biscuit]['washburn_eqn']
    storage_oftols[biscuit] = get_all_peak_tol_evolutions(datapoints, bw_values)

In [7]:
def kde_max_min_limits(datapoints):
    max_bin_density = sns.kdeplot(datapoints, bw_adjust=max_bin).get_lines()[0].get_data()
    min_bin_density = sns.kdeplot(datapoints, bw_adjust=min_bin).get_lines()[1].get_data()

    all_y_density = np.concatenate((max_bin_density[1], min_bin_density[1]))
    all_x_density = np.concatenate((max_bin_density[0], min_bin_density[0]))
    plt.close()
    
    return all_x_density, all_y_density

def tol_min_max_limits(biscuit_type):
    min_max = np.concatenate((storage_oftols[biscuit_type][min_tolerance], storage_oftols[biscuit_type][max_tolerance]))
    return min(min_max[:, 0]), max(min_max[:, 0])

In [8]:
def bandwidth_widget(biscuit_type = 'Digestive', bw=1, tolerance = 1e6):
    datapoints = dunky_df[dunky_df['biscuit'] == biscuit_type]['washburn_eqn']
    x_peaks, y_peaks = get_peaks(datapoints, bw, tolerance)
    all_x_density, all_y_density = kde_max_min_limits(datapoints)
    sns.kdeplot(datapoints, bw_adjust=bw, lw = 3, fill=True, label=F'KDE bw: {bw:.3f}')
    plt.scatter(x_peaks, y_peaks, color='red')
    
    plt.xlim(min(all_x_density)*0.995, max(all_x_density)*1.005)
    plt.ylim(0, max(all_y_density)*1.05)
    
    plt.xlabel('Average Pore Radius / m')
    
    plt.legend(loc='upper right', fontsize='small')
    plt.show()
    
interactive(bandwidth_widget, biscuit_type = ['Digestive', 'Hobnob', 'Rich Tea'], bw=(min_bin, max_bin, bw_steps), 
            tolerance=(FloatSlider(min=min_tolerance, max=max_tolerance, step=tolerance_steps, readout_format='.2e')))

interactive(children=(Dropdown(description='biscuit_type', options=('Digestive', 'Hobnob', 'Rich Tea'), value=…

In [10]:
def peak_evolution_widget(biscuit_type = 'Digestive', bw = 0.5, tolerance = 1e6):
    evolutions = storage_oftols[biscuit_type][tolerance]
    evolution_under_bw = evolutions[evolutions[:,1] <= bw]
    x_min, x_max = tol_min_max_limits(biscuit_type)

    plt.scatter(evolutions[:,0], evolutions[:,1], label=F'Tol: {tolerance:.2e}', alpha=0.5, color='grey')
    plt.scatter(evolution_under_bw[:,0], evolution_under_bw[:,1], label=F'Under BW: {bw:.2f}')
    
    plt.axhline(y=bw, color=palette[0], linestyle='--', linewidth=2.5)
    
    plt.xlim(x_min*0.995, x_max*1.005)
    plt.ylim(min(bw_values), max(bw_values)*1.05)
    
    plt.xlabel('Pore Radius Value/ m')
    plt.ylabel('Bandwidth Value')
    
    plt.legend(loc='upper right', fontsize='small')
    plt.show()
    
interactive(peak_evolution_widget, biscuit_type = ['Digestive', 'Hobnob', 'Rich Tea'], bw=(min_bin, max_bin, bw_steps),
            tolerance=(FloatSlider(min=min_tolerance, max=max_tolerance, step=tolerance_steps, readout_format='.2e')))

interactive(children=(Dropdown(description='biscuit_type', options=('Digestive', 'Hobnob', 'Rich Tea'), value=…